# MMLigner User Tutorial


#### Developed at Volkamerlab, Charité/FU Berlin 

by Dennis Köser

# Reference

Collier, J.H., Allison, L., Lesk A.M., Stuckey, P.J., Garcia de la Banda , M., Konagurthu, A.S. (2017)
Statistical inference of protein structural alignments using information and compression.
Bioinformatics, 33(7), 1005-1013

Wallace,C.S. and Boulton,D.M. (1968) An information measure for classification.
Comput. J., 11, 185–194.

Wallace,C.S. (2005) Statistical and Inductive Inference Using MinimumMessage Length.
Information Science and Statistics. SpringerVerlag, New York, NY.

Shannon,C.E. (1948) A mathematical theory of communication.
Bell Syst.Tech. J., 27, 379–423.

# Introduction


## What are the  chosen structures

Since this project was developed during the SARS-CoV-2 pandemic of 2020, we chose the main protease of SARS-Cov from 2006 (2GZ9) and the main protease of SARS-Cov 2 from 2020 (5R8T) as example structures for this tutorial. SARS-Cov and SARS-Cov 2 are strains of viruses that cause severe acute respiratory syndrome (SARS). The chosen proteases are required for the maturation of SARS-Cov and SARS-Cov 2 respectively, so they make a good target for structure-based drug design of anti-SARS drugs.

## Why they have been chosen

In addition to the relevance for the pandemic of 2020, these structures work well as an example for this tutorial because of their relation to each other resulting in quite similar but not completly identical structures. They both have a length of 306 and, if you align them with Needelman-Wunsch, show a similarity 98.69 % and identity of 96.08 %.

# Theory

## About MMLigner

MMLigner (Collier et al., 2017) works by minimizing the ivalue of the alignment. The ivalue is based on
the Minimum Message Length framework (Wallace and Boulton, 1968; Wallace, 2005), a Bayesian framework for
statistical inductive inference. The ivalue represents the hypothetical minimum message length needed to transmit
the computed alignment losslessly (Shannon, 1948).
Using the ivalue measure, the algorithm creates crude-but-effective strucural alignments rapidly to act as seeds.
These seeds are iteratively refined over an Expectation-Maximization approach using the I-value criterion.
By substracting the ivalue from the null model, the statistical significance of the alignment can be computed. If the
difference is positive, the alignment is significant.

## About the RCSB

The [RCSB PDB](http://www.rcsb.org/) (Research Collaboratory for Structural Bioinformatics Protein Data Bank) provide a global PDB archive, and makes PDB data available at no charge to all data consumers without limiations on usage.

## RMSD

The RMSD is the average distance between the atoms of tuperposed structures in Angstrom.

## coverage

The coverage of the aligned structures

## ivalue

The minimum message length of the compressed alignment in bites.

# Preperation

## How to get the structure from the CLI

To get the structures directly from the RCSB, the syntax looks like this:

In [1]:
!structuralalignment --method=mmligner 2GZ9 5R8T

   _____ _                   _                   _
  / ____| |                 | |                 | |
 | (___ | |_ _ __ _   _  ___| |_ _   _ _ __ __ _| |
  \___ \| __| '__| | | |/ __| __| | | | '__/ _` | |
  ____) | |_| |  | |_| | (__| |_| |_| | | | (_| | |
 |_____/ \__|_|   \__,_|\___|\__|\__,_|_|  \__,_|_|
     /\   | (_)                                | |
    /  \  | |_  __ _ _ __  _ __ ___   ___ _ __ | |_
   / /\ \ | | |/ _` | '_ \| '_ ` _ \ / _ \ '_ \| __|
  / ____ \| | | (_| | | | | | | | | |  __/ | | | |_
 /_/    \_\_|_|\__, |_| |_|_| |_| |_|\___|_| |_|\__|
                __/ |
               |___/

 v0+untagged.106.g1132d4c.dirty · Brought to you by @volkamerlab

RMSD for alignment between `2GZ9` and `5R8T` is 1.2Å


When you want to use structures which are locally saved, do this:

In [ ]:
!structuralalignment --method=mmligner PATH_OF_2GZ9 PATH_OF_5R8T

## Getting the structure in python

The method will use atomium.models as input.

If you want to get the structures from the RCSB, you can do the following:

In [2]:
import atomium

structure1 = atomium.fetch("2GZ9").model
structure2 = atomium.fetch("5R8T").model

# Alignment 

Using MMligner in Python looks like this:

In [3]:
from structuralalignment.superposition.mmligner import MMLignerAligner

mmligner = MMLignerAligner()
results = mmligner.calculate([structure1, structure2])

You can access the RMSD value like this:

In [4]:
results["scores"]["rmsd"]

1.244

Or the used transformation matrix like this:

In [5]:
results["metadata"]["rotation"]

[[0.257, 0.015, 0.966], [-0.02, 1.0, -0.01], [-0.966, -0.017, 0.257]]

The results are returned as a dictionary containing the superposed structures, the RMSD, ivalue and coverage, the alignment, the transformation matrix, the translation vector as well as the quarternion matrix. The dictionary looks like this:

In addition you can compute the ivalue of a already computed alignment. This looks like: 

In [6]:
alignment = results["metadata"]["alignment"]

results_ivalue = mmligner.ivalue([structure1, structure2], alignment)

results_ivalue["scores"]["score"]

20323.7

Because ivalue only scores an already computed alignment, only the scores are returned. The dictionary looks like this:

# Analysis

Lets look at the computed alignment!

### NGLview

If you have trouble with NGLview, follow this [troubleshooting guide](https://github.com/SBRG/ssbio/wiki/Troubleshooting#tips-for-nglview).

In [7]:
def atomium_to_nglview(*models):
    """
    Represent Atomium models in NGLView
    Parameters
    ----------
    models : atomium.Model
    Returns
    -------
    nglview.NGLWidget
    """
    import nglview as nv
    from structuralalignment.utils import enter_temp_directory

    v = nv.NGLWidget()
    with enter_temp_directory():
        for i, model in enumerate(models):
            fn = f"tmp{i}.pdb"
            model.save(fn)
            fs = nv.FileStructure(fn)
            v.add_component(fs)
    return v

In [8]:
import nglview as nv
print("nglview version = {}".format(nv.__version__))
# your nglview version should be 1.1.7 or later

v = atomium_to_nglview(*results["superposed"])
v

_ColormakerRegistry()

nglview version = 2.7.5


NGLWidget()

As expected the structures are a pretty good match.

## Report

Using MMLigner we get an alignment with a RMSD of 1.244Å. The alignment covers 303 of the 306 residues. The results show how closly related the two viruses are and may provide a opportunity to transfer research approaches from the development of anti-SARS drugs targeting the main protease of SARS-Cov to drugs targeting the the main protease of SARS-Cov 2.